In [1]:

import boto3

from boto_functions.ec2_functions import create_instance

ImportError: cannot import name 'logger' from partially initialized module 'boto_functions.ec2_functions' (most likely due to a circular import) (\\wsl.localhost\Ubuntu\home\avinash\Data-Science\User-Behaviour-Analytics\User-Behaviour-Analytics\boto_functions\ec2_functions.py)

In [2]:
s3_resource = boto3.resource('s3')

In [4]:
bucket = 'test-for-infra-script'

In [5]:

create_bucket(bucket_name=bucket, s3_resource=s3_resource, logger=logger)

{'ResponseMetadata': {'RequestId': 'P3NCJC69SA6KW5JQ',
  'HostId': 'bvg7V/obf0UNfavNkc1i+npPXYrusxCzNK1OXAvyyoql5wT6nSPR6AnKdwRB2XVqgV78oDs3eKQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bvg7V/obf0UNfavNkc1i+npPXYrusxCzNK1OXAvyyoql5wT6nSPR6AnKdwRB2XVqgV78oDs3eKQ=',
   'x-amz-request-id': 'P3NCJC69SA6KW5JQ',
   'date': 'Tue, 19 Jul 2022 07:21:26 GMT',
   'location': 'http://test-for-infra-script.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://test-for-infra-script.s3.amazonaws.com/'}

In [5]:
delete_bucket(bucket_name=bucket, s3_resource=s3_resource, logger=logger)

{'ResponseMetadata': {'RequestId': '1E6MX85V0WG8JSQ2',
  'HostId': 'Rp1Q1WevwcB1U8vFujQT41bYmStpLeg7AZi5VSq/3ESBeO8wJcuqFiN4WaYS7NFrH8h4oUVV00I=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Rp1Q1WevwcB1U8vFujQT41bYmStpLeg7AZi5VSq/3ESBeO8wJcuqFiN4WaYS7NFrH8h4oUVV00I=',
   'x-amz-request-id': '1E6MX85V0WG8JSQ2',
   'date': 'Tue, 19 Jul 2022 07:31:09 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [1]:
import socket
hostname = socket.gethostname()   
IPAddr = socket.gethostbyname(hostname)   
print("Your Computer Name is:" + hostname)   
print("Your Computer IP Address is:" + IPAddr)

Your Computer Name is:MSI
Your Computer IP Address is:127.0.1.1


In [10]:
#get my ip
import subprocess
ip = subprocess.run(['curl', '-s', 'http://whatismyip.akamai.com/'],stderr=subprocess.PIPE, text=True)
print(ip.stderr)

122.161.240.125


In [38]:
import subprocess
def my_ip():
    process = subprocess.Popen(['curl', '-s', 'http://whatismyip.akamai.com/'],stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    ip = process.communicate()
    return ip[0]

In [39]:
ip = my_ip()
ip

'122.161.240.125'

In [3]:
ip[0]

'122.161.240.125'

In [5]:
def attach_ingress(my_ip, security_group, group_id, description):
    cidr = my_ip+'/32'
    response = security_group.authorize_ingress(
        CidrIp=cidr,
        FromPort=23,
        GroupId= group_id,
        IpProtocol='tcp',
        ToPort=123 
    )

def attach_egress(my_ip, security_group):
    """ attaches egress rule, Port is hardcoded, free to edit"""
    cidr = my_ip+'/32'
    response = security_group.authorize_egress(
        IpPermissions=[
        {
            'FromPort': 8080,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': cidr,
                    'Description': 'my_ip'
                },
            ],
            'ToPort': 8080
        },
    ],
    )

In [9]:
ec2 = boto3.resource('ec2')

In [13]:

security_group = ec2.SecurityGroup('sg-0c7f8f5f8b15ee00c')



In [14]:
ec2_security_group_id = security_group.group_id
ec2_security_group_id

'sg-0c7f8f5f8b15ee00c'

In [18]:
from botocore.exceptions import ClientError
def create_instance(ec2, security_group_id, instance_profile_name, instance_type):
    """ creates an EC2 instance for our computing needs
    Hardcoded: instance number max and min values, OS image_id, volumetype, availability zone, 
    """
    try:
        instance = ec2.create_instances(
            BlockDeviceMappings=[
                {
                    'Ebs': {
                        'DeleteOnTermination': False,
                        'Iops': 200,
                        'VolumeSize': 8,
                        'VolumeType': 'gp2'
                    },
                },
            ],
            ImageId='ami-08df646e18b182346',
            InstanceType=instance_type,
            MaxCount=1,
            MinCount=1,
            Monitoring={
                'Enabled': False
            },
            SecurityGroupIds=[
                security_group_id
            ],
            IamInstanceProfile={
                'Name': instance_profile_name
            },
            
        )
    except ClientError:
        raise
    else:
        return instance

In [19]:

create_instance(ec2=ec2, security_group_id=ec2_security_group_id, instance_profile_name='ec2-role-project-sr1-19-7-2022-instance-profile', instance_type='t2.large')

ClientError: An error occurred (InvalidParameterValue) when calling the RunInstances operation: Invalid availability zone: [ap-south-1]